In [ ]:
!python3 -m pip install spacy
!python3 -m spacy download en
!python3 -m pip install gensim
!python3 -m pip install forbiddenfruit

In [2]:
# Path to mounted Gutenberg DVD.
gb_path = "/Volumes/PGDVD_2010_04_RC2"

In [4]:
metadf = parse_metadata(gb_path)
metadf.groupby("Author").count()["Title"].sort_values(ascending=False)

Author
Various                                           1937
                                                   761
Anonymous                                          517
Shakespeare, William, 1564-1616                    257
Lytton, Edward Bulwer Lytton, Baron, 1803-1873     212
Ebers, Georg, 1837-1898                            164
Twain, Mark, 1835-1910                             154
Parker, Gilbert, 1862-1932                         133
Balzac, Honoré de, 1799-1850                       123
Unknown                                            115
Kingston, William Henry Giles, 1814-1880           114
Jacobs, W. W. (William Wymark), 1863-1943          111
Meredith, George, 1828-1909                        109
Verne, Jules, 1828-1905                            104
Motley, John Lothrop, 1814-1877                    103
Howells, William Dean, 1837-1920                   100
Ballantyne, R. M. (Robert Michael), 1825-1894       98
Dickens, Charles, 1812-1870                         92
Haw

In [30]:
selected = filter_meta(metadf,author="Doyle")
txts = get_texts(selected, path=gb_path)

Loading EText 10446: 'The Green Flag' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 10581: 'Uncle Bernac
A Memory of the Empire' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 108: 'The Return of Sherlock Holmes' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 11247: 'The Exploits of Brigadier Gerard' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 11413: 'The Refugees' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 11656: 'The Great Shadow and Other Napoleonic Tales' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 12555: 'The Tragedy of the Korosko' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 126: 'The Poison Belt' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 13152: 'The Firm of Girdlestone' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 139: 'The Lost World' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 1638: 'The New Revelation' by Doyle, Arthur Conan, Sir, 1859-1930
Loading EText 1644: 'The Adventures of Gerard' by Doy

In [1]:
w2v_scores = {'capital-common-countries': 0.2490118577075099,
 'capital-world': 0.05106100795755968,
 'currency': 0.010392609699769052,
 'city-in-state': 0.08519793459552495,
 'family': 0.6126482213438735,
 'gram1-adjective-to-adverb': 0.29536290322580644,
 'gram2-opposite': 0.17364532019704434,
 'gram3-comparative': 0.8911411411411412,
 'gram4-superlative': 0.6729055258467023,
 'gram5-present-participle': 0.553030303030303,
 'gram6-nationality-adjective': 0.28267667292057536,
 'gram7-past-tense': 0.6230769230769231,
 'gram8-plural': 0.753003003003003,
 'gram9-plural-verbs': 0.542528735632184,
 'Total accuracy': 0.34704396680583477}